<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/gmnoh/Naive_bayes_classifier_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
import pandas as pd

In [56]:
# csv 만들기
data = ({'content':['me free lottery', 'free get free you', 'you free scholarship', 'free to contact me', 'you won award', 'you ticket lottery'], 
         'spam_normal':['spam', 'spam', 'normal', 'normal', 'normal', 'spam']})

df = pd.DataFrame(data)
df.to_csv('spam_mail.csv')

In [57]:
# csv 읽어오기
spam_data = pd.read_csv('spam_mail.csv', index_col = 0)
spam_data

,content,spam_normal
0,me free lottery,spam
1,free get free you,spam
2,you free scholarship,normal
3,free to contact me,normal
4,you won award,normal
5,you ticket lottery,spam


In [61]:
class spam_probability():
    def __init__(self, data, k, *keywords):
        self.data = data
        self.k = k
        self.keywords = keywords
        

    # 모든 mail에서 spam / noramal mail 분류
    def spam_or_not(self):
        self.spam = []
        self.normal = []

        for i in range(len(self.data['content'])):
            if self.data['spam_normal'][i] == 'spam':
                self.spam.append(self.data['content'][i])
            else:
                self.normal.append(self.data['content'][i])


    # spam / normal mail 확률
    def spam_normal_prob(self):
        self.prob_spam, self.prob_normal = len(self.spam) / len(self.data['content']), len(self.normal) / len(self.data['content'])
        self.log_spam, self.log_normal = np.log(self.prob_spam), np.log(self.prob_normal)


    # spam / normal mail 토큰화
    def spam_normal_tokening(self):
        self.spam = " ".join(self.spam)
        self.normal = " ".join(self.normal)

        self.spam_token = self.spam.split()
        self.normal_token = self.normal.split()

        self.tokens = list(set(self.spam_token).union(set(self.normal_token)))


    # spam / normal mail 단어별 log 확률
    def word_prob(self):
        self.spam_count = [self.spam_token.count(i) for i in self.tokens]
        self.normal_count = [self.normal_token.count(i) for i in self.tokens]

        self.log_w_spam = [np.log((i+self.k)/(2*self.k+sum(self.spam_count))) for i in self.spam_count]
        self.log_w_normal = [np.log((i+self.k)/(2*self.k+sum(self.normal_count))) for i in self.normal_count]


    def get_prob_with_word(self): 
        # spam / normal mail별 keyword(free, lottery) 토큰이 들어있을 확률
        sum  = 0
        for i in self.keywords[0]:
            sum += self.log_w_spam[self.tokens.index(i)]
        self.prob_word_spam = np.exp(sum + self.log_spam)
            
        sum  = 0
        for i in self.keywords[0]:
            sum += self.log_w_normal[self.tokens.index(i)]
        self.prob_word_normal = np.exp(sum + self.log_normal)

        # free, lottery가 포함된 메일이 spam / normal mail일 확률
        self.spam_result = self.prob_word_spam / (self.prob_word_normal + self.prob_word_spam)
        self.normal_result = self.prob_word_normal / (self.prob_word_normal + self.prob_word_spam)


    def get_spam_prob(self):
        self.spam_or_not()
        self.spam_normal_prob()
        self.spam_normal_tokening()
        self.word_prob()
        self.get_prob_with_word()

        return self.spam_result, self.normal_result


In [62]:
a = spam_probability(spam_data, 0.5, ['free', 'lottery'])

In [63]:
spam_prob, normal_prob = a.get_spam_prob()

print(spam_prob, normal_prob)

0.8749999999999999 0.12500000000000008
